In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from collections import Counter
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
import eli5

---

In [3]:
#!wget https://github.com/buriy/russian-nlp-datasets/releases/download/r4/lenta.tar.bz2

In [4]:
df = pd.read_csv('lenta.tar.bz2',usecols = ['topics','text'])
print(df.shape)
df.head()

(700006, 2)


,topics,text
0,Библиотека / Первая мировая,Бои у Сопоцкина и Друскеник закончились отступ...
1,Библиотека / Первая мировая,"Министерство народного просвещения, в виду про..."
2,Библиотека / Первая мировая,Фотограф-корреспондент Daily Mirror рассказыва...
3,Библиотека / Первая мировая,"Штабс-капитан П. Н. Нестеров на днях, увидев в..."
4,Библиотека / Первая мировая,"Лица, приехавшие в Варшаву из Люблина, передаю..."


In [5]:
df.dropna(inplace=True)
df.isnull().sum()

topics    0
text      0
dtype: int64

In [6]:
df.topics.unique().shape[0]

125

In [7]:
df.topics.value_counts().head()

Россия       116891
Мир           95099
Экономика     58247
Спорт         42323
Культура      34609
Name: topics, dtype: int64

In [8]:
take1st5topics = df.topics.value_counts().head(3).index
take1st5topics

Index(['Россия', 'Мир', 'Экономика'], dtype='object')

In [9]:
df = df[df.topics.isin(take1st5topics)]
df.shape

(270237, 2)

---

In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [11]:
le.fit(df.topics)
list(le.classes_)

['Мир', 'Россия', 'Экономика']

In [12]:
le.transform(df.topics) 

array([1, 1, 1, ..., 2, 2, 2])

In [13]:
le.transform(['Россия'])[0]

1

In [14]:
df['target'] = le.transform(df.topics)
df.drop(['topics'],axis=1,inplace=True)
df.reset_index(inplace=True,drop=True)

df.sample(5)

,text,target
131885,На одной из веток петербургского метро было пр...,1
238186,Банду подростков из подмосковной Балашихи запо...,1
3680,Новым руководителем Московского художественног...,1
36169,В четверг вечером вертолет израильских ВВС нан...,0
59948,"Татьяна Пичугина, супруга арестованного сотруд...",1


---

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(df.text,df.target,
                                                    test_size=0.2,
                                                    stratify=df.target,
                                                    random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((216189,), (54048,), (216189,), (54048,))

In [17]:
X_test.isna().sum()

0

---

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression 

### построим простую модельку без препроцессинга

In [19]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [20]:
%%time
cv = CountVectorizer(min_df=0.1)
cv_train = cv.fit_transform(X_train)
cv_test = cv.transform(X_test)

CPU times: user 29.5 s, sys: 347 ms, total: 29.8 s
Wall time: 29.8 s


In [21]:
cv.vocabulary_

{'компании': 40,
 'по': 65,
 'на': 50,
 'миллионов': 48,
 'долларов': 26,
 'что': 104,
 'от': 63,
 '30': 3,
 'до': 25,
 'процентов': 74,
 'году': 21,
 'то': 95,
 'же': 31,
 'время': 14,
 'года': 20,
 'еще': 30,
 'россии': 79,
 'за': 32,
 'он': 61,
 'из': 34,
 'частности': 102,
 'для': 24,
 'власти': 12,
 'сообщает': 85,
 'его': 27,
 'рф': 81,
 'они': 62,
 'президента': 70,
 'ранее': 75,
 'уже': 100,
 'заявил': 33,
 'против': 73,
 'об': 57,
 'этом': 108,
 'во': 13,
 'связи': 82,
 'не': 52,
 'было': 11,
 'решение': 77,
 'может': 49,
 'передает': 64,
 'которые': 42,
 'между': 47,
 'тем': 94,
 'напомним': 51,
 'как': 38,
 'будет': 6,
 'под': 66,
 'были': 10,
 'глава': 19,
 'был': 8,
 'словам': 83,
 'того': 96,
 'только': 97,
 'после': 68,
 'этого': 107,
 'сообщил': 87,
 'страны': 89,
 'более': 5,
 'результате': 76,
 'том': 98,
 'будут': 7,
 'ходе': 101,
 'которого': 41,
 'агентство': 4,
 'пока': 67,
 'сша': 91,
 'ее': 28,
 'однако': 59,
 'при': 72,
 'также': 93,
 'лет': 46,
 'интерфакс': 3

In [23]:
%%time
lr = LogisticRegression()
lr.fit(cv_train,y_train)

pred_lr = lr.predict(cv_test)
print('accuracy =',accuracy_score(y_test,pred_lr))
print('\nclassification_report:')
print(classification_report(y_test,pred_lr))
print('\nconfusion_matrix:')
print(confusion_matrix(y_test,pred_lr))

accuracy = 0.7625999111900533

classification_report:
              precision    recall  f1-score   support

           0       0.74      0.80      0.77     19020
           1       0.77      0.78      0.77     23378
           2       0.79      0.68      0.73     11650

   micro avg       0.76      0.76      0.76     54048
   macro avg       0.77      0.75      0.76     54048
weighted avg       0.76      0.76      0.76     54048


confusion_matrix:
[[15175  3146   699]
 [ 3913 18120  1345]
 [ 1448  2280  7922]]
CPU times: user 24.9 s, sys: 3.83 ms, total: 24.9 s
Wall time: 24.9 s


In [ ]:
#pipe = Pipeline([('vec', CountVectorizer(min_df=0.1)),
#                 ('clf', LogisticRegression())
#                 ]
#               )
#                  
#pipe.fit(X_train,y_train)
#
#pred_lr = pipe.predict(X_test)
#print('accuracy =',accuracy_score(y_test,pred_lr))
#print('\nclassification_report:')
#print(classification_report(y_test,pred_lr))
#print('\nconfusion_matrix:')
#print(confusion_matrix(y_test,pred_lr))

### построим простую модельку с препроцессингом

In [24]:
import re
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize

from razdel import tokenize # https://github.com/natasha/razdel
#!pip install razdel

import pymorphy2  # pip install pymorphy2

In [25]:
morph = pymorphy2.MorphAnalyzer()

In [26]:
stopword_ru = stopwords.words('russian')
len(stopword_ru)

with open('../stopwords.txt', 'r', encoding='utf-8') as f:
    for w in f.readlines():
        stopword_ru.append(w)
        
len(stopword_ru)

775

* clean_text

In [27]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
   
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')

    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())

    return text

In [28]:
df.head(2)

,text,target
0,В минувший четверг президент РФ Борис Ельцин п...,1
1,ИТАР-ТАСС со ссылкой на пресс-службу Миноборон...,1


In [29]:
%%time
df['clean_text'] = df.text.apply(clean_text)

CPU times: user 1min 10s, sys: 116 ms, total: 1min 11s
Wall time: 1min 11s


In [30]:
df.head(2)

,text,target,clean_text
0,В минувший четверг президент РФ Борис Ельцин п...,1,в минувший четверг президент рф борис ельцин п...
1,ИТАР-ТАСС со ссылкой на пресс-службу Миноборон...,1,итартасс со ссылкой на прессслужбу минобороны ...


* lemmatization

In [31]:
cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка на стоп-слова
        [4] проверка токена с одного символа
        [5] проверка есть ли данное слово в кэше
        [6] лемматизация слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    
    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if not w in stopword_ru: # [3]
            if len(w)>1: # [4]
                if w in cache: # [5]
                    words_lem.append(cache[w])
                else: # [6]
                    temp_cach = cache[w] = morph.parse(w)[0].normal_form
                    words_lem.append(temp_cach)
    return words_lem

In [ ]:
%%time
df['lemma_clean_text'] = df.text.apply(lemmatization)

In [ ]:
df.head(2)

In [ ]:
%%time
df['join_lemma_clean_text'] = df.lemma_clean_text.apply(lambda x: ' '.join(x))

In [ ]:
df.head(2)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.join_lemma_clean_text,df.target,
                                                    test_size=0.2,
                                                    stratify=df.target,
                                                    random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
X_test.isna().sum()

In [ ]:
%%time
cv = CountVectorizer(min_df=0.1)
cv_train = cv.fit_transform(X_train)
cv_test = cv.transform(X_test)

In [ ]:
cv.vocabulary_

In [ ]:
%%time
lr = LogisticRegression()
lr.fit(cv_train,y_train)

pred_lr = lr.predict(cv_test)
print('accuracy =',accuracy_score(y_test,pred_lr))
print('\nclassification_report:')
print(classification_report(y_test,pred_lr))
print('\nconfusion_matrix:')
print(confusion_matrix(y_test,pred_lr))